In [ ]:

    # yolo = YOLOHandler()

    # sam = SAM_handler(device = "cpu")
#input = "/home/michal/Documents/FIT/DP/dp/src/data/input/VID20240229170959.mp4"

In [ ]:
import cv2

In [ ]:
videoCap = cv2.VideoCapture("~/Documents/FIT/DP/dp/src/data/input/VID20240229170748.mp4")


In [ ]:
frame_num=0
while videoCap.isOpened():
    ret, frame = videoCap.read()
    bboxes, masks = self.frameProcessor.predict(frame)
    if frame_num > 0:
        cv2.namedWindow(f"{frame_num}", cv2.WINDOW_NORMAL)
                # Using resizeWindow()

        cv2.resizeWindow(f"{frame_num}", 1200, 800)
                # frameWithBboxes=frameWithBboxes[665:1400, 400:900]
        cv2.imshow(f"{frame_num}", frameWithBboxes)
        cv2.waitKey(0)
    frame_num+=1

In [ ]:
cap = cv2.VideoCapture("~/Documents/FIT/DP/dp/src/data/input/VID20240229170748.mp4")
if not cap.isOpened():
    print("Cannot open camera")
    exit()
while True:
 # Capture frame-by-frame
    ret, frame = cap.read()
 
 # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
 # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
 # Display the resulting frame
    cv2.imshow('frame', gray)
    if cv2.waitKey(1) == ord('q'):
        break
 
    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()


In [2]:
import numpy as np
import torch
import sys
sys.path.append("../../../")
# from src.impl.Frame_Processing.YOLO import YOLOHandler
# from src.impl.Frame_Processing.SAM import SAM_handler
from src.impl.Video_MTT_Manager import VideoMTT
from src.impl.MTT.PHD.PHD_tracker import PHDTracker
from src.impl.Frame_Processing import FrameProcessing
import cv2


def resize_masks(frameShape, masks):
        if masks is None or masks.shape[0] == 0:
            return None
        if masks.ndim == 3:
            if masks[0].shape != frameShape:
                new_masks = np.zeros(shape=(masks.shape[0], frameShape[1], frameShape[0]), dtype=np.uint8)
                for i, mask in enumerate(masks):
                    new_masks[i] = cv2.resize(mask, frameShape)
                return new_masks
            return masks.astype(np.uint8)

        elif masks.ndim == 4:
            new_masks = np.zeros(shape=(masks.shape[0], frameShape[1], frameShape[0]), dtype=np.uint8)
            if masks[0][0].T.shape != frameShape:
                masks = masks.transpose(1, 0, 2, 3)
                for i, mask in enumerate(masks[0]):
                    new_masks[i] = cv2.resize(mask, frameShape)
                return new_masks
            else:
                masks = masks.transpose(1, 0, 2, 3)
                return masks[0].astype(np.uint8)
        raise Exception("resize masks not supported")

def get_color(color):
        return int(color[0]), int(color[1]), int(color[2])

def add_color( mask, color):
        next_mask = mask.astype(np.uint8)
        next_mask = np.expand_dims(next_mask, 0).repeat(3, axis=0)
        next_mask = np.moveaxis(next_mask, 0, -1)
        return next_mask * color
        
        
def mergeMasks(masks, color = (0,0,128)):
        color = get_color(color)
        merged_masks = add_color(masks[0], color)
        if len(masks) == 1:
            return merged_masks.astype(np.uint8)
        for i in range(1, len(masks)):
            curr_colored_mask = add_color(masks[i], color)
            merged_masks = np.bitwise_or(merged_masks, curr_colored_mask)

        return merged_masks.astype(np.uint8)
        
        
        
def showAllMasks(masks, frame, color = (0,0,128)):
        masks = mergeMasks(masks, color)
        frame = cv2.addWeighted(frame, 1, masks, 0.8, 0)
        return frame

device = "cuda" if torch.cuda.is_available() else "cpu"
frameProcessor = FrameProcessing(mode=2, device=device)

 
# Creating a VideoCapture object to read the video
#cap = cv2.VideoCapture("~/Documents/FIT/DP/dp/src/data/input/VID20240229170748.mp4")
cap = cv2.VideoCapture("../../data/input/VID20240229170748.mp4.mp4")
frame_num=0
# Loop until the end of the video
while (cap.isOpened()):
 
    # Capture frame-by-frame
    ret, frame = cap.read()
   # frame = cv2.resize(frame, (540, 380), fx = 0, fy = 0,
    #                     interpolation = cv2.INTER_CUBIC)
 
    # Display the resulting frame
    if frame_num !=155:
        frame_num +=1
        print(frame_num)
        #cv2.imshow(f"{frame_num}",frame)
        continue
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    # conversion of BGR to grayscale is necessary to apply this operation
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
 
    # adaptive thresholding to use different threshold 
    # values on different regions of the frame.
    #Thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
     #                                      cv2.THRESH_BINARY_INV, 11, 2)
 
    #cv2.imshow('Thresh', Thresh)
    
    # define q as the exit button
    frameProcessor.set_frameDimensions((height, width))
    bboxes, masks = frameProcessor.predict(frame)
    frameShape =(width, height)
    print(masks.shape)
    masks = resize_masks(frameShape, masks)
    
    bboxes = bboxes.xyxy.numpy()
    for xyxy in bboxes:
        x1, y1, x2, y2 = int(xyxy[0]), int(xyxy[1]), int(xyxy[2]), int(xyxy[3])
        print(x1,y1,x2,y2)
        frame = cv2.rectangle(frame, (x1, y1), (x2, y2),color=(0,0,255), thickness=2)
   # print(bboxes.xyxy.numpy())
    #print("masks:")
    #print(masks)
    frame = showAllMasks(masks, frame)
    text="YOLO"
    cv2.namedWindow(f"{text}", cv2.WINDOW_NORMAL)
    cv2.resizeWindow(f"{text}", 800, 600)
    cv2.imshow(f"{text}", frame)
    cv2.waitKey(0)
 
# release the video capture object
cap.release()
# Closes all the windows currently opened.
cv2.destroyAllWindows()

/home/michal/Documents/FIT/DP/dp/src/impl/notebooks/../../../src/impl/Frame_Processing/DINO/__init__.py:12: UserWarning: GroundingDINO not available. Downloading Grounding DINO GIT repository
  warnings.warn("GroundingDINO not available. Downloading Grounding DINO GIT repository")
Cloning into 'GroundingDINO'...
mv: cannot move 'GroundingDINO' to '/home/michal/Documents/FIT/DP/dp/src/impl/notebooks/Frame_Processing/DINO/GroundingDINO': No such file or directory
error: pathspec '/home/michal/Documents/FIT/DP/dp/src/impl/notebooks/Frame_Processing/DINO/GroundingDINO' did not match any file(s) known to git
ERROR: /home/michal/Documents/FIT/DP/dp/src/impl/notebooks/Frame_Processing/DINO/GroundingDINO is not a valid editable requirement. It should either be a path to a local project or a VCS URL (beginning with bzr+http, bzr+https, bzr+ssh, bzr+sftp, bzr+ftp, bzr+lp, bzr+file, git+http, git+https, git+ssh, git+git, git+file, hg+file, hg+http, hg+https, hg+ssh, hg+static-http, svn+ssh, svn+h

FileNotFoundError: file "/home/michal/Documents/FIT/DP/dp/src/impl/notebooks/Frame_Processing/DINO/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py" does not exist